In [9]:
import sqlite3
import requests
import json
import datetime
from urllib.parse import urlparse
from urllib.parse import parse_qs
from time import sleep

# Load API Key
API_KEY = json.loads(open("secret.json").read())["api_key"]

# Connect to DB
conn = sqlite3.connect('f1_tumblr_analysis.db')
c = conn.cursor()

In [8]:
def getPostsToTimestamp(tag, timestamp = ""):
    if timestamp != "":
        x = requests.get("https://api.tumblr.com/v2/tagged?filter=raw&api_key="+API_KEY+"&tag="+tag+"&before="+str(timestamp))
    else:
        x = requests.get("https://api.tumblr.com/v2/tagged?filter=raw&api_key="+API_KEY+"&tag="+tag)
    return(x)

def processResponsePosts(APIResponse, replace = False, tag = None):
    
    
    # Get request tag
    if tag == None:
        tag = parse_qs(urlparse(APIResponse.request.url).query)['tag'][0]
    
    APIResponse = APIResponse.json()
    
    
    # Compute and print statistics about current batch of posts
    num_posts = len(APIResponse["response"])
    first_timestamp = APIResponse["response"][0]["timestamp"]
    last_timestamp = APIResponse["response"][-1]["timestamp"]
    print(
        first_timestamp,"->", last_timestamp,
        "(",round((first_timestamp-last_timestamp)/60,1), "min |",
        round(num_posts/(first_timestamp-last_timestamp)*60,1), "posts/min)\n"
    )
    
    # Iterate through each post
    for post in APIResponse["response"]:
        
        # Save Post Data
        post_content_raw = post['trail'][0]["content_raw"]
        post_url = post["post_url"]
        post_author = post["blog_name"]
        post_timestamp = post["timestamp"]
        post_id = post["id"]
        post_tags = post["tags"]
        
        print(post_timestamp,"\t", post_author,"\t", )
        
        # Check If Post Already in DB
        c.execute('select * from "' + tag + '" where id=?', (post_id,))
        
        if c.fetchone() != None:
            # Post already in DB
            
            # Delete records if replacing them
            if not replace:
                continue
            c.execute('delete from "' + tag + '" where id=?', (post_id,))
        
        
        # Add post to DB
        c.execute(
            'insert into "' + tag + '" values (?,?,?,?,?,?)',
            (post_id, post_timestamp, post_url, post_author, post_content_raw, str(post_tags))
        )
    
        conn.commit()
    
    return(last_timestamp)

processResponsePosts(getPostsToTimestamp("abu dhabi gp 2023", 1701010800), replace = True)

1701010797 -> 1701010621 ( 2.9 min | 6.8 posts/min)

1701010797 	 esteben-hur 	
1701010796 	 eirianerisdar 	
1701010788 	 honeylew 	
1701010770 	 sebvettelsv5 	
1701010762 	 lecchar 	
1701010762 	 valyrfia 	
1701010759 	 shippingcannons 	
1701010753 	 riotinyellow 	
1701010743 	 leclercloml 	
1701010727 	 meep-meep-richie 	
1701010716 	 garykingz 	
1701010708 	 vettely 	
1701010677 	 russellius 	
1701010667 	 isadora16aslani 	
1701010646 	 ro55ocorsa 	
1701010629 	 eightyones 	
1701010629 	 umgeorge 	
1701010625 	 multiseb21 	
1701010624 	 elenazamos 	
1701010621 	 rizz07 	


1701010621

In [11]:
def fetchPostsForRace(race, endTime = 1701010800, startTime = 1701003600, delay = 500):
    nextTimeStamp = endTime
    
    while (nextTimeStamp >= startTime):
    
        nextTimeStamp = processResponsePosts(
                                getPostsToTimestamp("abu dhabi gp 2023", nextTimeStamp),
                                replace = True
                        )
    sleep(delay / 1000)

fetchPostsForRace("abu dhabi gp 2023")

1701010797 -> 1701010621 ( 2.9 min | 6.8 posts/min)

1701010797 	 esteben-hur 	
1701010796 	 eirianerisdar 	
1701010788 	 honeylew 	
1701010770 	 sebvettelsv5 	
1701010762 	 lecchar 	
1701010762 	 valyrfia 	
1701010759 	 shippingcannons 	
1701010753 	 riotinyellow 	
1701010743 	 leclercloml 	
1701010727 	 meep-meep-richie 	
1701010716 	 garykingz 	
1701010708 	 vettely 	
1701010677 	 russellius 	
1701010667 	 isadora16aslani 	
1701010646 	 ro55ocorsa 	
1701010629 	 eightyones 	
1701010629 	 umgeorge 	
1701010625 	 multiseb21 	
1701010624 	 elenazamos 	
1701010621 	 rizz07 	
1701010612 -> 1701010442 ( 2.8 min | 7.1 posts/min)

1701010612 	 vroomvroomsposts 	
1701010611 	 sv5lh44 	
1701010590 	 umlewis 	
1701010584 	 vettely 	
1701010584 	 prettyimpossiblelady 	
1701010579 	 tabii-png 	
1701010578 	 slythereen 	
1701010574 	 racingcore 	
1701010573 	 valyrfia 	
1701010570 	 f1andindycars 	
1701010527 	 cooloddball 	
1701010525 	 leclerdelune 	
1701010517 	 wejustvibing 	
1701010513 	 dan

1701009389 -> 1701009329 ( 1.0 min | 20.0 posts/min)

1701009389 	 karlmarxverstappen 	
1701009388 	 serchive 	
1701009377 	 lestappenfiles 	
1701009375 	 caleb-is-existing 	
1701009375 	 sharedshield 	
1701009373 	 multiseb21 	
1701009371 	 hyacinthsdiamonds 	
1701009361 	 thenamesvettel 	
1701009358 	 andrewcogliano 	
1701009356 	 foul-milk 	
1701009354 	 riotinyellow 	
1701009351 	 keepthedelta 	
1701009351 	 esteville 	
1701009351 	 isadora16aslani 	
1701009348 	 mangoxangel 	
1701009341 	 usernyoom 	
1701009338 	 mynicosensesaretingling 	
1701009333 	 lleclercism 	
1701009329 	 scuderiacaps 	
1701009329 	 goregoal 	
1701009327 -> 1701009285 ( 0.7 min | 28.6 posts/min)

1701009327 	 formulalfc 	
1701009327 	 livetogether--diealone 	
1701009327 	 gloves-and-steeringwheel 	
1701009325 	 l4ndown-under 	
1701009322 	 f1andindycars 	
1701009320 	 leclercpiastri 	
1701009317 	 whoopsie-daisie 	
1701009316 	 leclercgoesnyooom 	
1701009309 	 slythereen 	
1701009300 	 leclercgoesnyooom 	
17

1701008476 -> 1701008409 ( 1.1 min | 17.9 posts/min)

1701008476 	 australiansteel 	
1701008471 	 leclerdelune 	
1701008468 	 spacesatori 	
1701008467 	 leclerking 	
1701008465 	 f1andindycars 	
1701008465 	 garykingz 	
1701008458 	 hookhausenschips 	
1701008454 	 pressedstresseddepressed 	
1701008452 	 alexalbn 	
1701008449 	 borntoraisekittys 	
1701008443 	 730clem 	
1701008443 	 umlewis 	
1701008436 	 keysmashhhhhhhh 	
1701008432 	 premium-deli-meats 	
1701008431 	 backwardscaplando 	
1701008428 	 ferrari-to-burnout-pipeline 	
1701008428 	 foul-milk 	
1701008424 	 monacotrophywife 	
1701008424 	 jaxgreen 	
1701008409 	 vroomvroomsposts 	
1701008408 -> 1701008326 ( 1.4 min | 14.6 posts/min)

1701008408 	 backwardscaplando 	
1701008400 	 sacharowan 	
1701008399 	 thisisyourdriverspeaking 	
1701008391 	 silverstoneddd 	
1701008388 	 spacesatori 	
1701008376 	 ellalovesvettel 	
1701008368 	 racingliners 	
1701008365 	 zaynzierulez 	
1701008358 	 umgeorge 	
1701008358 	 iguessricciardo 	

1701007332 -> 1701007179 ( 2.5 min | 7.8 posts/min)

1701007332 	 nuclear-sharks 	
1701007317 	 alexalbn 	
1701007305 	 umlewis 	
1701007301 	 oshiete 	
1701007299 	 yukinss 	
1701007299 	 mistressemmedi 	
1701007282 	 delicateglitch 	
1701007280 	 borntoraisekittys 	
1701007270 	 sv5lh44 	
1701007256 	 foul-milk 	
1701007252 	 esteben-hur 	
1701007251 	 mercgaydes 	
1701007250 	 princepiastri 	
1701007249 	 twinkodium 	
1701007232 	 riotinyellow 	
1701007224 	 umlewis 	
1701007217 	 punisherinthealps 	
1701007213 	 karlmarxverstappen 	
1701007198 	 bastet55 	
1701007179 	 agentstarkid 	
1701007177 -> 1701007039 ( 2.3 min | 8.7 posts/min)

1701007177 	 marsinout 	
1701007174 	 formulapisces 	
1701007170 	 hotwheelschili 	
1701007163 	 sebvettelsv5 	
1701007146 	 backwardscaplando 	
1701007143 	 valyrfia 	
1701007134 	 lh44jpg 	
1701007130 	 honeylew 	
1701007119 	 sv5lh44 	
1701007106 	 umlewis 	
1701007104 	 heartsoftruth 	
1701007098 	 lhpics7 	
1701007077 	 leclerking 	
1701007071 	

1701005510 -> 1701005483 ( 0.5 min | 44.4 posts/min)

1701005510 	 aston-angel 	
1701005510 	 architect-2015 	
1701005505 	 valyrfia 	
1701005504 	 altsplicing 	
1701005503 	 keysmashhhhhhhh 	
1701005502 	 keepthedelta 	
1701005502 	 talictries 	
1701005502 	 sblackbrekker 	
1701005502 	 whoopsie-daisie 	
1701005501 	 rednyx-sf 	
1701005501 	 lhpics7 	
1701005501 	 rustencohle 	
1701005500 	 brasiliangp 	
1701005499 	 bibibitch1308 	
1701005498 	 devinlefay 	
1701005490 	 sebvettelsv5 	
1701005490 	 sinnerism 	
1701005486 	 abovecalamity 	
1701005486 	 wafflesrisa 	
1701005483 	 ricciardo-is-racing 	
1701005482 -> 1701005461 ( 0.3 min | 57.1 posts/min)

1701005482 	 vroomvroomsposts 	
1701005482 	 karlmarxverstappen 	
1701005481 	 nyctes 	
1701005479 	 borntoraisekittys 	
1701005478 	 chriss-club 	
1701005477 	 keepthedelta 	
1701005477 	 lovelylunas-world 	
1701005474 	 leclercgoesnyooom 	
1701005474 	 secondseatcurse 	
1701005474 	 formula-niall 	
1701005472 	 twentyfourthstreet 	
17

1701004325 -> 1701004192 ( 2.2 min | 9.0 posts/min)

1701004325 	 twinkodium 	
1701004321 	 f1gozoom 	
1701004307 	 keepthedelta 	
1701004306 	 alb-back 	
1701004302 	 drivestraight 	
1701004286 	 umgeorge 	
1701004275 	 munira-s 	
1701004271 	 mrcusarmstrong 	
1701004269 	 leclerking 	
1701004265 	 wild-and-fluorescent-adolescent 	
1701004257 	 mercgaydes 	
1701004253 	 f1andindycars 	
1701004249 	 forzasedici 	
1701004227 	 garykingz 	
1701004227 	 leclerc-verstappen 	
1701004224 	 hotwheelschili 	
1701004203 	 l8tof1 	
1701004201 	 drivestraight 	
1701004199 	 umgeorge 	
1701004192 	 patrick-mahomes 	
1701004184 -> 1701004076 ( 1.8 min | 11.1 posts/min)

1701004184 	 valyrfia 	
1701004182 	 iguessricciardo 	
1701004180 	 jabber-2-by-2 	
1701004169 	 sinsthinkingof 	
1701004160 	 papayatifosi 	
1701004158 	 parallelplayers 	
1701004147 	 backwardscaplando 	
1701004145 	 jabber-2-by-2 	
1701004138 	 antisoucials 	
1701004132 	 umgeorge 	
1701004131 	 andrewcogliano 	
1701004125 	 born

In [ ]:
def raceSummary(race):
    
    
raceSummary("abu dhabi gp 2023")